# Data Center Cooling Optimization Engine
## Phase 1 - Rule-Based Chiller Load Reduction Recommendation

**Objective**:  
Use the dataset to build a Python engine that:
- Checks rack thermal safety
- Detects peak tariff hours
- Recommends safe chiller load reduction with cost savings estimation

**NOTE**: No plotting libraries used, as per restrictions.

**Deliverables**:
- Data ingestion & validation
- Thermal safety check
- Peak tariff detection (combines tariff value and time of day)
- Cooling reduction decision engine
- Savings estimation
- Output table (CSV + displayed)
- Sample data generation
- README section

In [9]:
# Cell 1: Imports and Configuration (NO PLOTTING)
import pandas as pd
import numpy as np
import warnings
import os
from datetime import datetime

# Configuration (easily adjustable)
CFG = {
    "tariff_csv": "mumbai_weekly_tariffs.csv",           # Tariff file
    "data_csv": "data_center_sample.csv",                # Input data (generated if missing)
    "output_csv": "cooling_optimization_output.csv",     # Final output
    "peak_tariff_threshold": 7.0,                        # ₹/kWh → Consider PEAK if >= this
    "off_peak_tariff_threshold": 5.0,                    # ₹/kWh → Consider OFF_PEAK if <= this
    "peak_hours_start": 8,                               # Peak time start (hour, 24h format)
    "peak_hours_end": 20,                                # Peak time end (hour)
    "off_peak_hours_start": 22,                          # Off-peak start (for night hours)
    "off_peak_hours_end": 6,                             # Off-peak end (wraps around midnight)
    "reduction_min_pct": 5,                              # Min reduction %
    "reduction_max_pct": 10,                             # Max reduction %
}

print("Configuration loaded. No plotting libraries imported.")

Configuration loaded. No plotting libraries imported.


In [10]:
# Cell 2: Generate Sample Data (if real data not provided)
def generate_sample_data(num_hours=168):
    """Generate realistic synthetic data for testing."""
    np.random.seed(42)
    
    timestamps = pd.date_range("2025-12-23 00:00", periods=num_hours, freq="H", tz="Asia/Kolkata")
    
    tariff = 4.5 + 3.5 * np.sin(2 * np.pi * (timestamps.hour - 6) / 24)
    tariff = np.clip(tariff, 4.0, 9.0)
    
    rack_inlet = 24 + 4 * np.random.normal(size=num_hours)
    rack_outlet = rack_inlet + 8 + 2 * np.random.normal(size=num_hours)
    
    chiller_power = 140 + 30 * np.sin(2 * np.pi * (timestamps.hour - 12) / 24) + np.random.normal(0, 10, num_hours)
    
    df = pd.DataFrame({
        "timestamp": timestamps,
        "tariff": np.round(tariff, 2),
        "rack_inlet_temp": np.round(rack_inlet, 2),
        "rack_outlet_temp": np.round(rack_outlet, 2),
        "chiller_power_kw": np.round(np.clip(chiller_power, 80, 200), 2)
    })
    
    # Introduce invalid rows for validation testing
    df.loc[0, "rack_inlet_temp"] = -5
    df.loc[1, "chiller_power_kw"] = -10
    df.loc[2, "rack_outlet_temp"] = np.nan
    
    return df

if not os.path.exists(CFG["data_csv"]):
    print("Sample data not found. Generating synthetic data...")
    sample_df = generate_sample_data()
    sample_df.to_csv(CFG["data_csv"], index=False)
    print(f"Sample data saved to {CFG['data_csv']}")
else:
    print(f"Using existing data: {CFG['data_csv']}")

Using existing data: data_center_sample.csv


In [11]:
# Cell 3: Data Ingestion & Validation
def load_and_validate_data():
    df = pd.read_csv(CFG["data_csv"], parse_dates=["timestamp"])
    initial_count = len(df)
    print(f"Loaded {initial_count} rows.")
    
    # Merge tariff if not present
    if "tariff" not in df.columns:
        tariff_df = pd.read_csv(CFG["tariff_csv"])
        tariff_df["date"] = pd.to_datetime(tariff_df["Date"])
        tariff_df["start_hour"] = tariff_df["Time_Slot"].str.split("-").str[0].str[:2].astype(int)
        tariff_df["timestamp"] = tariff_df["date"] + pd.to_timedelta(tariff_df["start_hour"], unit="h")
        hourly = []
        for _, r in tariff_df.iterrows():
            idx = pd.date_range(r["timestamp"], freq="h", periods=6, tz="Asia/Kolkata")
            hourly.append(pd.DataFrame({"timestamp": idx, "tariff": r["Tariff_(₹/kWh)"]}))
        tariff_hourly = pd.concat(hourly).drop_duplicates("timestamp").set_index("timestamp")
        df = df.merge(tariff_hourly["tariff"], left_on="timestamp", right_index=True, how="left")
    
    # Validation
    missing_rows = df.isnull().any(axis=1)
    if missing_rows.sum() > 0:
        warnings.warn(f"Skipping {missing_rows.sum()} rows with missing values.")
        df = df[~missing_rows]
    
    temp_invalid = (
        (df["rack_inlet_temp"] < 0) | (df["rack_inlet_temp"] > 50) |
        (df["rack_outlet_temp"] < 0) | (df["rack_outlet_temp"] > 50)
    )
    if temp_invalid.sum() > 0:
        warnings.warn(f"Removing {temp_invalid.sum()} rows with invalid temperatures.")
        df = df[~temp_invalid]
    
    power_invalid = df["chiller_power_kw"] < 0
    if power_invalid.sum() > 0:
        warnings.warn(f"Removing {power_invalid.sum()} rows with negative power.")
        df = df[~power_invalid]
    
    df = df.set_index("timestamp").sort_index()
    print(f"After validation: {len(df)} rows remaining.")
    return df

df = load_and_validate_data()
df.head()

Loaded 168 rows.
After validation: 165 rows remaining.


C:\Users\HP\AppData\Local\Temp\ipykernel_27084\1481915894.py:23: UserWarning: Skipping 1 rows with missing values.
  warnings.warn(f"Skipping {missing_rows.sum()} rows with missing values.")
C:\Users\HP\AppData\Local\Temp\ipykernel_27084\1481915894.py:31: UserWarning: Removing 1 rows with invalid temperatures.
  warnings.warn(f"Removing {temp_invalid.sum()} rows with invalid temperatures.")
C:\Users\HP\AppData\Local\Temp\ipykernel_27084\1481915894.py:36: UserWarning: Removing 1 rows with negative power.
  warnings.warn(f"Removing {power_invalid.sum()} rows with negative power.")


,tariff,rack_inlet_temp,rack_outlet_temp,chiller_power_kw
timestamp,,,,
2025-12-23 03:00:00+05:30,4.00,30.09,36.46,113.15
2025-12-23 04:00:00+05:30,4.00,23.06,30.91,105.80
2025-12-23 05:00:00+05:30,4.00,23.06,31.75,113.46
2025-12-23 06:00:00+05:30,4.50,30.32,38.87,112.45
2025-12-23 07:00:00+05:30,5.41,27.07,36.72,105.95


In [12]:
# Cell 4: Thermal Safety Check
def check_thermal_safety(df):
    df = df.copy()
    df["delta_temp"] = df["rack_outlet_temp"] - df["rack_inlet_temp"]
    df["delta_temp_rise"] = df["delta_temp"].diff()
    
    df["safety_status"] = "SAFE"
    unsafe = (df["rack_inlet_temp"] > 30) | (df["delta_temp_rise"] > 2)
    df.loc[unsafe, "safety_status"] = "UNSAFE"
    
    print(f"Safety status summary:\n{df['safety_status'].value_counts().to_dict()}")
    return df

df = check_thermal_safety(df)
df[["rack_inlet_temp", "delta_temp", "delta_temp_rise", "safety_status"]].head(10)

Safety status summary:
{'SAFE': 115, 'UNSAFE': 50}


,rack_inlet_temp,delta_temp,delta_temp_rise,safety_status
timestamp,,,,
2025-12-23 03:00:00+05:30,30.09,6.37,NaN,UNSAFE
2025-12-23 04:00:00+05:30,23.06,7.85,1.48,SAFE
2025-12-23 05:00:00+05:30,23.06,8.69,0.84,SAFE
2025-12-23 06:00:00+05:30,30.32,8.55,-0.14,UNSAFE
2025-12-23 07:00:00+05:30,27.07,9.65,1.10,SAFE
2025-12-23 08:00:00+05:30,22.12,8.03,-1.62,SAFE
2025-12-23 09:00:00+05:30,26.17,10.91,2.88,UNSAFE
2025-12-23 10:00:00+05:30,22.15,7.47,-3.44,SAFE
2025-12-23 11:00:00+05:30,22.14,13.44,5.97,UNSAFE


In [13]:
# Cell 5: Peak Tariff Detection (Combines tariff value and time of day)
def detect_tariff_class(df):
    df = df.copy()
    df["hour"] = df.index.hour
    
    # Time-based conditions (configurable)
    is_peak_time = (df["hour"] >= CFG["peak_hours_start"]) & (df["hour"] < CFG["peak_hours_end"])
    is_off_peak_time = (df["hour"] >= CFG["off_peak_hours_start"]) | (df["hour"] < CFG["off_peak_hours_end"])
    
    # Classify combining value and time
    df["tariff_class"] = "NORMAL"
    df.loc[(df["tariff"] >= CFG["peak_tariff_threshold"]) | is_peak_time, "tariff_class"] = "PEAK"
    df.loc[(df["tariff"] <= CFG["off_peak_tariff_threshold"]) & is_off_peak_time, "tariff_class"] = "OFF_PEAK"
    
    print(f"Tariff class summary:\n{df['tariff_class'].value_counts().to_dict()}")
    return df

df = detect_tariff_class(df)
df[["tariff", "hour", "tariff_class"]].head(10)

Tariff class summary:
{'PEAK': 84, 'OFF_PEAK': 53, 'NORMAL': 28}


,tariff,hour,tariff_class
timestamp,,,
2025-12-23 03:00:00+05:30,4.00,3,OFF_PEAK
2025-12-23 04:00:00+05:30,4.00,4,OFF_PEAK
2025-12-23 05:00:00+05:30,4.00,5,OFF_PEAK
2025-12-23 06:00:00+05:30,4.50,6,NORMAL
2025-12-23 07:00:00+05:30,5.41,7,NORMAL
2025-12-23 08:00:00+05:30,6.25,8,PEAK
2025-12-23 09:00:00+05:30,6.97,9,PEAK
2025-12-23 10:00:00+05:30,7.53,10,PEAK
2025-12-23 11:00:00+05:30,7.88,11,PEAK


In [14]:
# Cell 6: Cooling Reduction Decision Engine
def recommend_reduction(df):
    df = df.copy()
    baseline_kw = df["chiller_power_kw"].mean()
    print(f"Baseline chiller power: {baseline_kw:.2f} kW")
    
    condition = (
        (df["safety_status"] == "SAFE") &
        (df["tariff_class"] == "PEAK") &
        (df["chiller_power_kw"] > baseline_kw)
    )
    
    df["recommendation"] = "NO"
    df.loc[condition, "recommendation"] = "YES"
    
    reductions = np.random.uniform(CFG["reduction_min_pct"], CFG["reduction_max_pct"], size=len(df))
    df["suggested_reduction_pct"] = 0.0
    df.loc[condition, "suggested_reduction_pct"] = reductions[condition]
    
    print(f"Recommendations:\n{df['recommendation'].value_counts().to_dict()}")
    return df

df = recommend_reduction(df)
df[["safety_status", "tariff_class", "chiller_power_kw", "recommendation", "suggested_reduction_pct"]].head(10)

Baseline chiller power: 140.14 kW
Recommendations:
{'NO': 129, 'YES': 36}


,safety_status,tariff_class,chiller_power_kw,recommendation,suggested_reduction_pct
timestamp,,,,,
2025-12-23 03:00:00+05:30,UNSAFE,OFF_PEAK,113.15,NO,0.0
2025-12-23 04:00:00+05:30,SAFE,OFF_PEAK,105.80,NO,0.0
2025-12-23 05:00:00+05:30,SAFE,OFF_PEAK,113.46,NO,0.0
2025-12-23 06:00:00+05:30,UNSAFE,NORMAL,112.45,NO,0.0
2025-12-23 07:00:00+05:30,SAFE,NORMAL,105.95,NO,0.0
2025-12-23 08:00:00+05:30,SAFE,PEAK,109.31,NO,0.0
2025-12-23 09:00:00+05:30,UNSAFE,PEAK,121.11,NO,0.0
2025-12-23 10:00:00+05:30,SAFE,PEAK,110.52,NO,0.0
2025-12-23 11:00:00+05:30,UNSAFE,PEAK,118.16,NO,0.0


In [15]:
# Cell 7: Savings Estimation
def estimate_savings(df):
    df = df.copy()
    df["estimated_kw_saved"] = df["chiller_power_kw"] * df["suggested_reduction_pct"] / 100
    df["estimated_rs_saved"] = df["estimated_kw_saved"] * df["tariff"]
    
    daily_savings = df["estimated_rs_saved"].resample("D").sum()
    total = daily_savings.sum()
    
    print("Daily savings (₹):")
    print(daily_savings.round(2))
    print(f"\nTotal estimated savings: ₹{total:.2f}")
    return df

df = estimate_savings(df)

Daily savings (₹):
timestamp
2025-12-23 00:00:00+05:30    395.87
2025-12-24 00:00:00+05:30    473.07
2025-12-25 00:00:00+05:30    348.81
2025-12-26 00:00:00+05:30    315.34
2025-12-27 00:00:00+05:30    399.77
2025-12-28 00:00:00+05:30    269.73
2025-12-29 00:00:00+05:30    335.44
Freq: D, Name: estimated_rs_saved, dtype: float64

Total estimated savings: ₹2538.03


In [16]:
# Cell 8: Output Generator (Fixed KeyError by resetting index before column selection)
def generate_output(df):
    cols = ["timestamp", "recommendation", "suggested_reduction_pct",
            "estimated_kw_saved", "estimated_rs_saved", "safety_status"]
    
    # Reset index first to make 'timestamp' a column
    out_df = df.reset_index()[cols].copy()
    
    out_df["timestamp"] = out_df["timestamp"].dt.strftime("%Y-%m-%d %H:%M")
    out_df["suggested_reduction_pct"] = out_df["suggested_reduction_pct"].round(1).astype(str) + "%"
    out_df["estimated_kw_saved"] = out_df["estimated_kw_saved"].round(1)
    out_df["estimated_rs_saved"] = out_df["estimated_rs_saved"].round(1)
    
    out_df.to_csv(CFG["output_csv"], index=False)
    print(f"Output saved to {CFG['output_csv']}")
    display(out_df.head(10))
    
    return out_df

output_df = generate_output(df)

Output saved to cooling_optimization_output.csv


,timestamp,recommendation,suggested_reduction_pct,estimated_kw_saved,estimated_rs_saved,safety_status
0,2025-12-23 03:00,NO,0.0%,0.0,0.0,UNSAFE
1,2025-12-23 04:00,NO,0.0%,0.0,0.0,SAFE
2,2025-12-23 05:00,NO,0.0%,0.0,0.0,SAFE
3,2025-12-23 06:00,NO,0.0%,0.0,0.0,UNSAFE
4,2025-12-23 07:00,NO,0.0%,0.0,0.0,SAFE
5,2025-12-23 08:00,NO,0.0%,0.0,0.0,SAFE
6,2025-12-23 09:00,NO,0.0%,0.0,0.0,UNSAFE
7,2025-12-23 10:00,NO,0.0%,0.0,0.0,SAFE
8,2025-12-23 11:00,NO,0.0%,0.0,0.0,UNSAFE
9,2025-12-23 12:00,NO,0.0%,0.0,0.0,SAFE


In [ ]:
# README - Project Overview

**Data Center Cooling Optimization Engine (Phase 1)**

### Features Implemented:
- **Data Ingestion**: Loads CSV, parses timestamps, validates temperatures (0-50°C), removes negative power and missing values with warnings.
- **Thermal Safety**: Inlet ≤30°C, ΔT rise ≤2°C per interval (assumes data interval matches rule; adjust for 15-min data if needed).
- **Peak Detection**: Combines tariff thresholds and configurable time-of-day windows for PEAK/NORMAL/OFF_PEAK.
- **Decision Engine**: Recommends 5-10% reduction only if SAFE, PEAK, and power > baseline (mean).
- **Savings**: kW saved = power * pct/100; ₹ saved = kW saved * tariff (per hour, duration=1); daily totals printed.
- **Output**: CSV in exact format, plus displayed table.

### Assumptions:
- Hourly data (adjust diff for 15-min if needed).
- Baseline = mean chiller power.
- Tariff + time combined: PEAK if high tariff OR peak hours; OFF_PEAK if low tariff AND off-peak hours.

### Files:
- `data_center_sample.csv` → input (auto-generated if missing)
- `cooling_optimization_output.csv` → recommendations

### Config:
All thresholds/time windows in `CFG` dictionary.

**Fully compliant: Python, pandas, numpy only. No ML/cloud/dashboards/plotting.**

Ready for review/handover.